In [1]:
import os
os.chdir('..')

In [2]:
import cudf

In [7]:
clean_cdr = cudf.read_parquet('data/processed/201308/clean_cdr')
meta_tower = cudf.read_csv('data/processed/meta_tower.csv')

meta_tower

,cell_id,lon,lat,deyang_center_flag,tower_area_code
0,0001,103.993,30.7214,0,28
1,0002,103.993,30.7214,0,28
2,0003,103.993,30.7214,0,28
3,0004,104.161,30.6291,0,28
4,0005,104.161,30.6291,0,28
...,...,...,...,...,...
40564,FFE8,103.966,30.5730,0,28
40565,FFF0,103.980,30.6016,0,28
40566,FFF2,103.966,30.5730,0,28
40567,FFF4,104.066,30.5781,0,28


In [8]:
N = len(clean_cdr)
clean_cdr

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id
__null_dask_index__,,,,,,,,,,
0,9afmdpyv,ejra2suz,1,5,ejra2suz,838,53,838,4ofzl4yr,3CC3
1,9afmdpyv,ejra2suz,1,5,ejra2suz,838,27,838,4ofzl4yr,3CC3
2,9aqm8dgb,gh8w8ki0,1,5,9pzs3exd,838,61,838,gh8w8ki0,351D
3,a2fug2rm,ftik422s,1,5,ftik422s,838,53,838,931s2hft,36B8
4,a2fug2rm,ftik422s,1,5,ftik422s,838,45,838,931s2hft,36B7
...,...,...,...,...,...,...,...,...,...,...
44148431,ad0i8l8y,3tps5cty,10,3,3tps5cty,838,10,838,joxo5s46,37F1
44148432,a5aied44,a1hc5coh,10,3,k04s2qxm,838,48,838,a1hc5coh,392D
44148433,9akyedck,hyz45c7x,10,3,hyz45c7x,838,27,838,2gyvjxty,37FA


In [9]:
group1 = clean_cdr[
    (clean_cdr['calling_area_code'] == 838) &
    (clean_cdr['called_area_code'] != 838)
].copy()
n1 = len(group1)

group2 = clean_cdr[
    (clean_cdr['calling_area_code'] != 838) &
    (clean_cdr['called_area_code'] == 838)
].copy()
n2 = len(group2)

print(f'total number oberservations in cdr: {N}\n')


n_ = len(clean_cdr[
    (clean_cdr['calling_area_code'] == 838) &
    (clean_cdr['called_area_code'] == 838)
])
print(f'total number of observations which both calling and called are local: {n_}')
print(f'percentage: {n_ / N * 100}%\n')

print(f'total number of observation in group1: {n1}')
print(f'percentage: {n1 / N * 100}%\n')

print(f'total number of observation in group2: {n2}')
print(f'percentage: {n2 / N * 100}%')

total number oberservations in cdr: 52840417

total number of observations which both calling and called are local: 44410897
percentage: 84.04721143665464%

total number of observation in group1: 4492391
percentage: 8.501808378991408%

total number of observation in group2: 3937129
percentage: 7.450980184353957%


# Client number?

In [10]:
print('percentage of observations in cdr which client_nbr is calling_nbr: ')
print(f"{(clean_cdr['client_nbr'] == clean_cdr['calling_nbr']).sum() / N * 100}%")

percentage of observations in cdr which client_nbr is calling_nbr: 
53.62071423471166%


In [11]:
print('percentage of observations in cdr which client_nbr is called_nbr: ')
print(f"{(clean_cdr['client_nbr'] == clean_cdr['called_nbr']).sum() / N * 100}%")

percentage of observations in cdr which client_nbr is called_nbr: 
46.37951097168669%


In [12]:
print('percentage of observations in group1 which client_nbr is calling_nbr: ')
print(f"{(group1['client_nbr'] == group1['calling_nbr']).sum() / n1 * 100}%")

percentage of observations in group1 which client_nbr is calling_nbr: 
100.0%


In [13]:
print('percentage of observations in group2 which client_nbr is called_nbr: ')
print(f"{(group2['client_nbr'] == group2['called_nbr']).sum() / n2 * 100}%")

percentage of observations in group2 which client_nbr is called_nbr: 
100.0%


# Experiment - group1

In [14]:
group1_ = cudf.merge(group1, meta_tower, on='cell_id')

group1_

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id,lon,lat,deyang_center_flag,tower_area_code
0,al13jq8y,itjzfvjh,10,5,itjzfvjh,838,40,28,l3693726,36FC,104.661,31.0963,0,838
1,a3a19o7p,3q3tjdgm,10,5,3q3tjdgm,838,28,28,9ag9iccl,059B,103.981,30.6195,0,28
2,9q6d8l61,joplbd9u,10,5,joplbd9u,838,70,28,8r9g7fr9,0EFE,104.230,30.8952,0,28
3,9fg84aug,cxcml31f,10,5,cxcml31f,838,132,839,kh0v5tcm,46A0,105.977,31.8428,0,839
4,apqw4b04,fhxyl370,10,5,fhxyl370,838,55,28,fied2ync,3851,104.395,31.1299,1,838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309618,98ev3ojv,3c52kn2a,19,3,3c52kn2a,838,142,28,e85tfzuz,0B3C,103.485,30.4816,0,28
4309619,9f7v2deg,h00tk0gk,19,3,h00tk0gk,838,51,816,l4gg5frq,2A0A,104.481,31.6085,0,816
4309620,9f7v2deg,h00tk0gk,19,3,h00tk0gk,838,79,816,l6k95kh6,2A0A,104.481,31.6085,0,816
4309621,a2t5ayqq,71e049k4,19,3,71e049k4,838,366,816,dm53jrnv,3490,104.340,31.0552,0,838


In [17]:
print('case when cell location is the same with calling:')
n1_ = len(group1_[group1_['calling_area_code'] == group1_['tower_area_code']])
print(n1_)
print(f'{n1_ / 4309623 * 100}%')

case when cell location is the same with calling:
3094028
71.79347242206569%


In [19]:
print('case when cell location is the same with called:')
n2_ = len(group1_[group1_['called_area_code'] == group1_['tower_area_code']])
print(n2_)
print(f'{n2_ / 4309623 * 100}%')

case when cell location is the same with called:
892070
20.699490419463604%


In [20]:
print('remained:')
print(f'{(1 - n1_ / 4309623 - n2_ / 4309623) * 100}%')

remained:
7.507037158470712%


# Experiment - group2

In [21]:
group2_ = cudf.merge(group2, meta_tower, on='cell_id')

group2_

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id,lon,lat,deyang_center_flag,tower_area_code
0,9ez3m2yy,n8843a2j,21,5,l4kojjve,28,723,838,n8843a2j,388D,104.845,30.6924,0,838
1,9al12d3d,nfuk3aav,21,5,2r3168e2,28,44,838,nfuk3aav,0860,104.082,30.6838,0,28
2,aa8jd3a6,k44x2k56,21,5,4s39hk13,660,54,838,k44x2k56,37BA,104.395,31.1685,0,838
3,9ndmblw0,kws2kusa,21,5,dwq5lq6r,28,237,838,kws2kusa,3799,104.349,31.0299,0,838
4,a5liel51,e1gskn57,22,5,5rinnfoj,28,29,838,e1gskn57,02BA,103.801,30.6434,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781632,9748cv11,3q10lxcy,11,3,kv7227ya,816,115,838,3q10lxcy,354D,104.392,31.1282,1,838
3781633,a2d7hdre,lu73f07r,11,3,2f9qjw4u,28,76,838,lu73f07r,3537,104.186,30.9302,0,838
3781634,9uxsbm1g,itx12c71,11,3,ez8ac29r,28,37,838,itx12c71,3683,104.585,30.8653,0,838
3781635,9fanhdan,8c33f0fw,11,3,fara3ly8,28,58,838,8c33f0fw,1F2E,104.011,30.6442,0,28


In [23]:
print('case when cell location is the same with calling:')
n1_ = len(group2_[group2_['calling_area_code'] == group2_['tower_area_code']])
print(n1_)
print(f'{n1_ / 3781637 * 100}%')

case when cell location is the same with calling:
715058
18.90868954370819%


In [25]:
print('case when cell location is the same with called:')
n2_ = len(group2_[group2_['called_area_code'] == group2_['tower_area_code']])
print(n2_)
print(f'{n2_ / 3781637 * 100}%')

case when cell location is the same with called:
2768451
73.20774045737335%


In [26]:
print('remained:')
print(f'{(1 - n1_ / 3781637 - n2_ / 3781637) * 100}%')

remained:
7.88356999891846%
